# This is my notebook for the segmenting Toronto neighborhoods assignment

### Part I: Scraping data

In [229]:
#needed packages
import pandas as pd
import geocoder
from sklearn.cluster import KMeans
import folium
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

pd.options.mode.chained_assignment = None

In [230]:
#read in the table from wikipedia
toronto_neighborhoods = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)

In [231]:
#format the dataframe
neighborhoods_df = toronto_neighborhoods[0]

neighborhoods_df = neighborhoods_df[neighborhoods_df.Borough != 'Not assigned']
neighborhoods_df['Neighbourhood'].replace('Not assigned', neighborhoods_df['Borough'], inplace=True)
neighborhoods_df = neighborhoods_df.groupby(['Postcode', 'Borough'], sort=False).agg(lambda x:', '.join(x))
neighborhoods_df.reset_index(level=['Postcode','Borough'], inplace=True)
neighborhoods_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


### Part II: Adding latitude and longitude

In [232]:
#add latitude and longitude coordinates to the dataframe
lat_long = pd.read_csv("Geospatial_Coordinates.csv", header=0)
lat_long.rename(columns={"Postal Code": "Postcode"}, inplace=True)
neighborhoods_df = neighborhoods_df.merge(lat_long, on = 'Postcode')
neighborhoods_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


### Part III: Segmenting and clustering

First, I decided to visualize the neighborhoods in Toronto to make sure everything looked correct. the map displays each postal code and hovering over the dots will display which neighborhoods belong to a given postal code.

In [233]:
#latitude and longitude of Toronto, Canada
tor_lat = 43.6532
tor_long = -79.3832

#create a folium map
map_toronto = folium.Map(location=[tor_lat, tor_long], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods_df['Latitude'], neighborhoods_df['Longitude'], neighborhoods_df['Borough'], neighborhoods_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

Now to segment and cluster the neighborhoods. To do so, I used data from Foursquare.

In [234]:
CLIENT_ID = 'XFVHDSOL1NLUBDL3FLWKTZE3E5P2XCVTFK1GVVA4Q0TC52LI' #Foursquare ID
CLIENT_SECRET = 'S4H4QOZMW2EFDC5IUICJL5OIZ0LONVIIQRHM2FQEHGJSEMN4' #Foursquare Secret
VERSION = '20180605' #Foursquare API version

#function to get venues near a given neighborhood (lat and long)
def getNearbyVenues(names, latitudes, longitudes, limit=100, radius=1000):
    
    venues_list = []
    for name, lat, long in zip(names, latitudes, longitudes):
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
         # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [235]:
#use the function to get a list of nearby venues for all the neighborhoods in Toronto
toronto_venues = getNearbyVenues(names = neighborhoods_df['Neighbourhood'],
                                latitudes = neighborhoods_df['Latitude'],
                                longitudes = neighborhoods_df['Longitude'])

toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.520999,Best Western Plus Toronto North York Hotel & S...,43.756887,-79.528779,Hotel
1,Parkwoods,43.753259,-79.520999,Commisso Bakery,43.748678,-79.517262,Tea Room
2,Parkwoods,43.753259,-79.520999,Shoppers Drug Mart,43.756147,-79.515843,Pharmacy
3,Parkwoods,43.753259,-79.520999,Holiday Inn Express Toronto-North York,43.756232,-79.527348,Hotel
4,Parkwoods,43.753259,-79.520999,Pizza Hut,43.756340,-79.517818,Pizza Place


In [250]:
#see how many venues I got for each neighborhoods
venues_count = toronto_venues.groupby('Neighborhood').count()
venues_count.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",53,53,53,53,53,53
Berczy Park,77,77,77,77,77,77
"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",75,75,75,75,75,75
"Brockton, Exhibition Place, Parkdale Village",50,50,50,50,50,50
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",56,56,56,56,56,56


Upon examining how many venues were found for each neighborhood, I noticed that many of the neighborhoods had only a few venues, making it (in my opinion) not very useful to use them in the clustering. Because of this, I decided to arbitrarily only include nieghborhoods with 50+ venues in my clustering.

In [251]:
venues_count = venues_count[venues_count['Venue'] > 49]
venues_count = venues_count.reset_index()
toronto_venues = toronto_venues[toronto_venues['Neighborhood'].isin(venues_count['Neighborhood'])]

Now I have my final list of neighborhoods to use for my segmenting and clustering, it's time to get on with it.

In [252]:
#one hot code the dataframe
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Adult Boutique,American Restaurant,Asian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Beer Store,Breakfast Spot,...,Supplement Shop,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Turkish Restaurant,Wings Joint,Yoga Studio
235,"Cloverdale, Islington, Martin Grove, Princess ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
236,"Cloverdale, Islington, Martin Grove, Princess ...",0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
237,"Cloverdale, Islington, Martin Grove, Princess ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
238,"Cloverdale, Islington, Martin Grove, Princess ...",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
239,"Cloverdale, Islington, Martin Grove, Princess ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group neighborhoods and take the mean frequency from each of the venue categories to create a "profile" of each neighborhood.

In [282]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Adult Boutique,American Restaurant,Asian Restaurant,Automotive Shop,BBQ Joint,Bakery,Bank,Beer Store,Breakfast Spot,...,Supplement Shop,Sushi Restaurant,Tanning Salon,Tapas Restaurant,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Turkish Restaurant,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.037736,0.018868,0.018868,0.018868,...,0.000000,0.056604,0.000000,0.018868,0.037736,0.000000,0.0,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.012987,0.025974,0.000000,0.012987,0.012987,...,0.000000,0.051948,0.000000,0.012987,0.025974,0.012987,0.0,0.012987,0.000000,0.012987
2,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0.000000,0.000000,0.000000,0.000000,0.013333,0.013333,0.000000,0.013333,0.000000,...,0.000000,0.040000,0.000000,0.013333,0.026667,0.013333,0.0,0.013333,0.000000,0.013333
3,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.020000,0.020000,0.000000,...,0.020000,0.000000,0.020000,0.000000,0.020000,0.000000,0.0,0.000000,0.020000,0.020000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.017857,0.035714,0.017857,0.017857,0.035714,0.035714,0.017857,0.000000,0.000000,...,0.017857,0.035714,0.017857,0.000000,0.017857,0.000000,0.0,0.000000,0.017857,0.035714


Create a dataframe that displays the top 10 most common venues for each neighborhood. This will be helpful in interpreting the clusters.

In [283]:
#returns the x most common venues in a given neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#create the dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Italian Restaurant,Sushi Restaurant,Pub,Fast Food Restaurant,Pizza Place,Thai Restaurant,Bakery,Gym,Metro Station
1,Berczy Park,Coffee Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Pub,Fast Food Restaurant,Thai Restaurant,Café,French Restaurant,Bakery
2,"Bloordale Gardens, Eringate, Markland Wood, Ol...",Coffee Shop,Italian Restaurant,Pizza Place,Pub,Fast Food Restaurant,Sushi Restaurant,Convenience Store,Pharmacy,Sandwich Place,Park
3,"Brockton, Exhibition Place, Parkdale Village",Bakery,Coffee Shop,Park,Fast Food Restaurant,Gym,Convenience Store,Pub,Sandwich Place,Grocery Store,Supplement Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Gym / Fitness Center,Burrito Place,Yoga Studio,Burger Joint,Italian Restaurant,Restaurant,Sandwich Place,Coffee Shop,Convenience Store,Bakery


Run the clustering algorithm on the neighborhoods. I messed around with the number of clusters until I found something that I thought was realistic and interpretable.

In [284]:
# set number of clusters
kclusters = 4

#cluster it
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [285]:
#add cluster labels to dataframe
neighborhoods_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",3,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pub,Fast Food Restaurant,Pizza Place,Thai Restaurant,Bakery,Gym,Metro Station
1,Berczy Park,3,Coffee Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Pub,Fast Food Restaurant,Thai Restaurant,Café,French Restaurant,Bakery
2,"Bloordale Gardens, Eringate, Markland Wood, Ol...",3,Coffee Shop,Italian Restaurant,Pizza Place,Pub,Fast Food Restaurant,Sushi Restaurant,Convenience Store,Pharmacy,Sandwich Place,Park
3,"Brockton, Exhibition Place, Parkdale Village",2,Bakery,Coffee Shop,Park,Fast Food Restaurant,Gym,Convenience Store,Pub,Sandwich Place,Grocery Store,Supplement Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",1,Gym / Fitness Center,Burrito Place,Yoga Studio,Burger Joint,Italian Restaurant,Restaurant,Sandwich Place,Coffee Shop,Convenience Store,Bakery


In [286]:
#add latitude and longitudes
toronto_merged = neighborhoods_df[neighborhoods_df['Neighbourhood'].isin(venues_count['Neighborhood'])]
toronto_merged.rename(columns={"Neighbourhood": "Neighborhood"}, inplace=True)

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724,0,Coffee Shop,Pizza Place,Sushi Restaurant,Pub,Fast Food Restaurant,Italian Restaurant,Thai Restaurant,Grocery Store,Gym,Bakery
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Pizza Place,Pub,Fast Food Restaurant,Italian Restaurant,Thai Restaurant,Grocery Store,Gym,Sushi Restaurant,Sandwich Place
17,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201,3,Coffee Shop,Italian Restaurant,Pizza Place,Pub,Fast Food Restaurant,Sushi Restaurant,Convenience Store,Pharmacy,Sandwich Place,Park
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Pub,Fast Food Restaurant,Thai Restaurant,Café,French Restaurant,Bakery
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,3,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pub,Fast Food Restaurant,Pizza Place,Thai Restaurant,Bakery,Gym,Metro Station


Here I visualized the resulting clusters on a folium map

In [287]:
# create map
map_clusters = folium.Map(location=[tor_lat, tor_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine the clusters to try to analyze the clusters and determine what type of neighborhood is in each

In [288]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"Cloverdale, Islington, Martin Grove, Princess ...",0,Coffee Shop,Pizza Place,Sushi Restaurant,Pub,Fast Food Restaurant,Italian Restaurant,Thai Restaurant,Grocery Store,Gym,Bakery
15,St. James Town,0,Coffee Shop,Pizza Place,Pub,Fast Food Restaurant,Italian Restaurant,Thai Restaurant,Grocery Store,Gym,Sushi Restaurant,Sandwich Place
36,"Harbourfront East, Toronto Islands, Union Station",0,Coffee Shop,Pizza Place,Pub,Park,Fast Food Restaurant,Convenience Store,Gym,Grocery Store,Yoga Studio,Gastropub
81,"Runnymede, Swansea",0,Coffee Shop,Pizza Place,Italian Restaurant,Pub,Fast Food Restaurant,Thai Restaurant,Grocery Store,Sushi Restaurant,Gym,Sandwich Place
84,"Chinatown, Grange Park, Kensington Market",0,Coffee Shop,Fast Food Restaurant,Pizza Place,Pub,Italian Restaurant,Sandwich Place,Pharmacy,Grocery Store,Sushi Restaurant,Gym
98,"The Kingsway, Montgomery Road, Old Mill North",0,Coffee Shop,Fast Food Restaurant,Pizza Place,Pub,Pharmacy,Grocery Store,Sushi Restaurant,Gym,Japanese Restaurant,Spa


In [289]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
87,"CN Tower, Bathurst Quay, Island airport, Harbo...",1,Gym / Fitness Center,Burrito Place,Yoga Studio,Burger Joint,Italian Restaurant,Restaurant,Sandwich Place,Coffee Shop,Convenience Store,Bakery
102,"Kingsway Park South West, Mimico NW, The Queen...",1,Gym / Fitness Center,Coffee Shop,Burrito Place,Yoga Studio,Burger Joint,Italian Restaurant,Restaurant,Sandwich Place,Convenience Store,Bakery


In [290]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,"Brockton, Exhibition Place, Parkdale Village",2,Bakery,Coffee Shop,Park,Fast Food Restaurant,Gym,Convenience Store,Pub,Sandwich Place,Grocery Store,Supplement Shop
76,Canada Post Gateway Processing Centre,2,Bakery,Coffee Shop,Park,Fast Food Restaurant,Gym,Convenience Store,Pub,Sandwich Place,Grocery Store,Supplement Shop


In [291]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,"Bloordale Gardens, Eringate, Markland Wood, Ol...",3,Coffee Shop,Italian Restaurant,Pizza Place,Pub,Fast Food Restaurant,Sushi Restaurant,Convenience Store,Pharmacy,Sandwich Place,Park
20,Berczy Park,3,Coffee Shop,Pizza Place,Sushi Restaurant,Italian Restaurant,Pub,Fast Food Restaurant,Thai Restaurant,Café,French Restaurant,Bakery
30,"Adelaide, King, Richmond",3,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pub,Fast Food Restaurant,Pizza Place,Thai Restaurant,Bakery,Gym,Metro Station
37,"Little Portugal, Trinity",3,Coffee Shop,Sushi Restaurant,Pizza Place,Pub,Fast Food Restaurant,Italian Restaurant,Gym,Thai Restaurant,Bakery,Metro Station
42,"Design Exchange, Toronto Dominion Centre",3,Coffee Shop,Sushi Restaurant,Pizza Place,Fast Food Restaurant,Pub,Italian Restaurant,French Restaurant,Sandwich Place,Grocery Store,Thai Restaurant
48,"Commerce Court, Victoria Hotel",3,Coffee Shop,Sushi Restaurant,Fast Food Restaurant,Pizza Place,Pub,Italian Restaurant,Gym,Thai Restaurant,Metro Station,Bakery
75,"Parkdale, Roncesvalles",3,Coffee Shop,Italian Restaurant,Pizza Place,Pub,Sushi Restaurant,Fast Food Restaurant,Gym,Thai Restaurant,Bakery,Metro Station
92,Stn A PO Boxes 25 The Esplanade,3,Coffee Shop,Sushi Restaurant,Pizza Place,Fast Food Restaurant,Pub,Italian Restaurant,French Restaurant,Sandwich Place,Grocery Store,Thai Restaurant
97,"First Canadian Place, Underground city",3,Coffee Shop,Sushi Restaurant,Fast Food Restaurant,Pizza Place,Pub,Italian Restaurant,Gym,Thai Restaurant,Metro Station,Bakery


### Interpretation

So the final results are a little muddled by coffee shops haha, but here are what labels I would ascribe to each cluster after examining the most common venues in each.

Cluster 0: Pizza and pub cluster
Cluster 1: Fitness cluster
Cluster 2: Bakery and park cluster
Cluster 3: Restaurant (especially Italian and sushi) cluster

Based off these clusters, I do think it's possible to look for a nieghborhood that has the activities important to you!